In [ ]:
# Test section, we take 600036 as an example
import tushare as ts
import time
from datetime import date
from datetime import timedelta

# initialize the score as 0
score = 0

# parameter, change total time span as you wish, 365 is a full year data
span = 365

# if you want to get the score of another specific date, change td to a specific date
td = date.today()
d = timedelta(days = span)
sd = td - d
start_date = sd.strftime("%Y-%m-%d")
today = td.strftime("%Y-%m-%d")

# 600036 招商银行
# 600050 中国联通
# 002157,601789,002547,002300
df = ts.get_k_data('002300', start = start_date, end = today)

# return last 13 weekdays data
df_ma_13d = df.iloc[-13:]
# return last 55 weekdays data
df_ma_55d = df.iloc[-55:]

# CONDITION 1
td_close = float(df_ma_13d['close'].iloc[-1:])
ma_13d = df_ma_13d['close'].mean()
if td_close > ma_13d:
    score = score + 1
else:
    score = score - 1
print("close price today is {}, and 13 days MA is {}".format(td_close , ma_13d))
print("after condition 1, current score is {}".format(score))

# CONDITION 2
# close price of today are the same, so we don't have to calculate it again
ma_55d = df_ma_55d['close'].mean()
if td_close > ma_55d:
    score = score + 1
else:
    score = score - 1
print("close price today is {}, and 55 days MA is {}".format(td_close , ma_55d))
print("after condition 2, current score is {}".format(score))

# CONDITION 3
# WE RECORD STH HERE, IF WE CAN SAVE 13DAY MA OF EVERYDAY, DO QUERY, ONLY CALCULATE WHEN THE VALUE IS NULL
prv_ma_13d = df['close'].iloc[-14:-1].mean()
if ma_13d > prv_ma_13d:
    score = score + 1
else:
    score = score - 1
print("Previous 13 MA is {}, current 13 MA is {}".format(prv_ma_13d, ma_13d))
print("after condition 3, current score is {}".format(score))

# CONDITION 4
prv_ma_55d = df['close'].iloc[-56:-1].mean()
if ma_55d > prv_ma_55d:
    score = score + 1
else:
    score = score - 1
print("Previous 55 MA is {}, current 55 MA is {}".format(prv_ma_55d, ma_55d))
print("after condition 4, current score is {}".format(score))

# CONDITION 5
df_kline = df_ma_55d
i = -1
j = 0
x = 0

hist_3d_h = df_kline['high'].iloc[-4:-1].max()
hist_3d_l = df_kline['low'].iloc[-4:-1].min()

print("highest price of last 3 days is {}, lowest price of last 3 days is {}"
      .format(hist_3d_h, hist_3d_l))

td_h = float(df_kline['high'].iloc[-1:])
td_l = float(df_kline['low'].iloc[-1:])
td_c = float(df_kline['close'].iloc[-1:])

print("highest price of today is {}, lowest price of today is {}, close price of today is {}"
      .format(td_h, td_l, td_c))

while x <= 3:
    # only highest
    if td_h > hist_3d_h and td_l >= hist_3d_l:
        score = score + 1
        break
    # only lowest
    elif td_h <= hist_3d_h and td_l < hist_3d_l:
        score = score - 1
        break
    # highest and lowest
    elif td_h > hist_3d_h and td_l < hist_3d_l:
        #what if equal???
        if abs(td_h - td_c) < abs(td_l - td_c):
            score = score + 1
            break
        else:
            score = score - 1
            break
    # neither highest nor lowest
    elif td_h <= hist_3d_h and td_l >= hist_3d_l:
        i = i - 1
        j = j - 1
        td_h = float(df_kline['high'].iloc[i:j])
        td_l = float(df_kline['low'].iloc[i:j])
        td_c = float(df_kline['close'].iloc[i:j])
        hist_3d_h = df_kline['high'].iloc[i-3:j-1].max()
        hist_3d_l = df_kline['low'].iloc[i-3:j-1].min()
        x = x + 1
print("after condition 5, current score is {}".format(score))
        
# CONDITION 6
i = -1
j = 0
x = 0

hist_15d_h = df_kline['high'].iloc[-16:-1].max()
hist_15d_l = df_kline['low'].iloc[-16:-1].min()

print("highest price of last 3 weeks is {}, lowest price of last 3 weeks is {}"
      .format(hist_15d_h, hist_15d_l))

td_h = float(df_kline['high'].iloc[-1:])
td_l = float(df_kline['low'].iloc[-1:])
td_c = float(df_kline['close'].iloc[-1:])

print("highest price of today is {}, lowest price of today is {}, close price of today is {}"
      .format(td_h, td_l, td_c))

while x <= 15:
    # highest
    if td_h > hist_15d_h and td_l >= hist_15d_l:
        score = score + 1
        break
    # lowest
    elif td_h <= hist_15d_h and td_l < hist_15d_l:
        score = score - 1
        break
    # highest and lowest
    elif td_h > hist_15d_h and td_l < hist_15d_l:
        #what if equal???
        if abs(td_h - td_c) < abs(td_l - td_c):
            score = score + 1
            break
        else:
            score = score - 1
            break
    # neither highest nor lowest
    elif td_h <= hist_15d_h and td_l >= hist_15d_l:
        i = i - 1
        j = j - 1
        td_h = float(df_kline['high'].iloc[i:j])
        td_l = float(df_kline['low'].iloc[i:j])
        td_c = float(df_kline['close'].iloc[i:j])
        hist_15d_h = df_kline['high'].iloc[i-15:j-1].max()
        hist_15d_l = df_kline['low'].iloc[i-15:j-1].min()
        x = x + 1
        continue
        
print("after condition 6, current score is {}".format(score))

# CONDITION 7
df_sz50 = ts.get_k_data('sz50', start = start_date, end = today)

ema_12_day = df_sz50['close'].ewm(span = 12, min_periods = 0, adjust = True, ignore_na = True).mean()
ema_26_day = df_sz50['close'].ewm(span = 26, min_periods = 0, adjust = True, ignore_na = True).mean()

df_sz50['12ema'] = ema_12_day
df_sz50['26ema'] = ema_26_day
df_sz50['DIFF'] = (df_sz50['12ema'] - df_sz50['26ema'])

# DEA calculation
dea = df_sz50['DIFF'].ewm(span = 9, min_periods = 0, adjust = True, ignore_na = True).mean()
df_sz50['DEA'] = dea
df_sz50['MACD'] = 2 * (df_sz50['DIFF'] - df_sz50['DEA'])

if float(df_sz50['MACD'].iloc[-1:]) > 0:
    score = score + 1
else:
    score = score - 1
print("after condition 7, current score is {}".format(score))